In [17]:
import sys
sys.path.append('/host/d/Github')
import nibabel as nb
import glob
import os
import glob
import lpips
import torch
import numpy as np
import pandas as pd
from skimage.metrics import structural_similarity

import Diffusion_denoising_thin_slice.functions_collection as ff
import Diffusion_denoising_thin_slice.Build_lists.Build_list as Build_list
import Diffusion_denoising_thin_slice.Data_processing as Data_processing

In [18]:
build_sheet =  Build_list.Build_thinsliceCT(os.path.join('/host/d/Data/brain_CT/Patient_lists/fixedCT_static_simulation_train_test_gaussian_xjtlu.xlsx'))
_,patient_id_list,patient_subid_list,random_num_list, condition_list, x0_list = build_sheet.__build__(batch_list = [5]) 
n = ff.get_X_numbers_in_interval(total_number = patient_id_list.shape[0],start_number = 0,end_number = 1, interval = 2)

In [19]:
def calc_mae_with_ref_window(img, ref, vmin, vmax):
    maes = []
    for slice_num in range(0, img.shape[-1]):
        slice_img = img[:,:,slice_num]
        slice_ref = ref[:,:,slice_num]
        mask = np.where((slice_ref >= vmin) & (slice_ref <= vmax), 1, 0)
        mae = np.sum(np.abs(slice_img - slice_ref) * mask) / np.sum(mask)
        maes.append(mae)

    return np.mean(maes), np.std(maes)

In [20]:
def calc_ssim_with_ref_window(img, ref, vmin, vmax):

    ssims = []
    for slice_num in range(0, img.shape[-1]):
        slice_img = img[:,:,slice_num]
        slice_ref = ref[:,:,slice_num]
        mask = np.where((slice_ref >= vmin) & (slice_ref <= vmax), 1, 0)
        _, ssim_map = structural_similarity(slice_img, slice_ref, data_range=vmax - vmin, full=True)
        ssim = np.sum(ssim_map * mask) / np.sum(mask)
        ssims.append(ssim)

    return np.mean(ssims), np.std(ssims)

In [21]:
def calc_lpips(imgs1, imgs2, vmin, vmax):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    loss_fn = lpips.LPIPS().to(device)
    
    lpipss = []
    for slice_num in range(0, imgs1.shape[-1]):
        slice1 = imgs1[:,:,slice_num]
        slice2 = imgs2[:,:,slice_num]

        slice1 = np.clip(slice1, vmin, vmax).astype(np.float32)
        slice2 = np.clip(slice2, vmin, vmax).astype(np.float32)

        slice1 = (slice1 - vmin) / (vmax - vmin) * 2 - 1
        slice2 = (slice2 - vmin) / (vmax - vmin) * 2 - 1

        slice1 = np.stack([slice1, slice1, slice1], axis=-1)
        slice2 = np.stack([slice2, slice2, slice2], axis=-1)
        # print('after stack, slice1 shape:', slice1.shape, ' slice2 shape:', slice2.shape)

        slice1 = np.transpose(slice1, (2, 0, 1))[np.newaxis, ...]
        slice2 = np.transpose(slice2, (2, 0, 1))[np.newaxis, ...]
        # print('after transpose, slice1 shape:', slice1.shape, ' slice2 shape:', slice2.shape)

        slice1 = torch.from_numpy(slice1).to(device)
        slice2 = torch.from_numpy(slice2).to(device)

        lpips_val = loss_fn(slice1, slice2)
        lpipss.append(lpips_val.item())

      

    return np.mean(lpipss), np.std(lpipss)


### postprocess noise2score

In [16]:
# results = []
# for i in range(0,n.shape[0]):
#     patient_id = patient_id_list[n[i]]
#     patient_subid = patient_subid_list[n[i]]
#     random_n = random_num_list[n[i]]
#     print(patient_id, patient_subid, random_n)

#     # reference image
#     gt_file = os.path.join('/host/d/projects/denoising/models/unsupervised_gaussian_brainCT/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch61_1/gt_img.nii.gz')
#     gt_img = nb.load(gt_file).get_fdata()

#     # noisy image
#     condition_file = os.path.join('/host/d/projects/denoising/models/unsupervised_gaussian_brainCT/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch61_1/condition_img.nii.gz')
#     condition_img = nb.load(condition_file).get_fdata()

#     # noise2noise
#     noise2noise_file = os.path.join('/host/d/projects/denoising/models/noise2noise_brainCT/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch78/pred_img.nii.gz')
#     noise2noise_img = nb.load(noise2noise_file).get_fdata() 

#     # noise2score
#     noise2score_file = os.path.join('/host/d/projects/denoising/models/noise2score_brainCT/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch100/pred_img.nii.gz')
#     noise2score_img = nb.load(noise2score_file).get_fdata()

#     ## calculate main
#     x,y = 256,256
#     mean_gt = np.mean(np.clip(gt_img[x-50: x+50, y-50: y+50, 20:40],0,100))
#     mean_condition = np.mean(np.clip(condition_img[x-50: x+50, y-50: y+50, 20:40],0,100))
#     mean_noise2noise = np.mean(np.clip(noise2noise_img[x-50: x+50, y-50: y+50, 20:40],0,100))
#     mean_noise2score = np.mean(np.clip(noise2score_img[x-50: x+50, y-50: y+50, 20:40],0,100))

#     print('mean: gt:', mean_gt, ' condition:', mean_condition, ' noise2noise:', mean_noise2noise, ' noise2score:', mean_noise2score)

#     delta = mean_noise2noise - mean_noise2score

#     noise2score_img += delta
#     affine = nb.load(noise2score_file).affine
#     nb.save(nb.Nifti1Image(noise2score_img, affine), os.path.join('/host/d/projects/denoising/models/noise2score_brainCT/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch100/pred_img_postprocess.nii.gz'))

In [ ]:
## metric calculations
from genericpath import isfile
import shutil

results = []
save_collection = True
for i in range(0,n.shape[0]):
    patient_id = patient_id_list[n[i]]
    patient_subid = patient_subid_list[n[i]]
    random_n = random_num_list[n[i]]

    print(patient_id, patient_subid, random_n)

    save_folder = os.path.join('/host/d/projects/denoising/models/pred_collections/brain_CT', patient_id, patient_subid)
    ff.make_folder([os.path.dirname(save_folder), save_folder])

    # reference image
    gt_file = os.path.join('/host/d/projects/denoising/models/unsupervised_gaussian_brainCT/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch61_1/gt_img.nii.gz')
    gt_img = nb.load(gt_file).get_fdata()
    if save_collection:
        shutil.copy(gt_file, os.path.join(save_folder, 'gt_img.nii.gz'))

    # noisy image
    condition_file = os.path.join('/host/d/projects/denoising/models/unsupervised_gaussian_brainCT/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch61_1/condition_img.nii.gz')
    condition_img = nb.load(condition_file).get_fdata()
    if save_collection:
        shutil.copy(condition_file, os.path.join(save_folder, 'condition_img.nii.gz'))

    # noise2noise
    noise2noise_file = os.path.join('/host/d/projects/denoising/models/noise2noise_brainCT/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch78/pred_img.nii.gz')
    noise2noise_img = nb.load(noise2noise_file).get_fdata() 
    if save_collection:
        shutil.copy(noise2noise_file, os.path.join(save_folder, 'noise2noise_img.nii.gz'))

    # noise2score
    noise2score_file = os.path.join('/host/d/projects/denoising/models/noise2score_brainCT/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch100/pred_img_postprocess.nii.gz')
    noise2score_img = nb.load(noise2score_file).get_fdata()
    if save_collection:
        shutil.copy(noise2score_file, os.path.join(save_folder, 'noise2score_img.nii.gz'))

    # DDM2_firststep
    ddm2_first_file = os.path.join('/host/d/projects/denoising/models/DDM2_brainCT/pred_images', patient_id, patient_subid,'ddm2_first_step.nii.gz')
    ddm2_first_img = nb.load(ddm2_first_file).get_fdata()
    if save_collection:
        shutil.copy(ddm2_first_file, os.path.join(save_folder, 'ddm2_firststep_img.nii.gz'))

    # DDM2_finalstep
    ddm2_final_file = os.path.join('/host/d/projects/denoising/models/DDM2_brainCT/pred_images', patient_id, patient_subid,'ddm2_final.nii.gz')
    ddm2_final_img = nb.load(ddm2_final_file).get_fdata()
    if save_collection:
        shutil.copy(ddm2_final_file, os.path.join(save_folder, 'ddm2_finalstep_img.nii.gz'))

    # supervised method
    supervised_file = os.path.join('/host/d/projects/denoising/models/supervised_poisson_brainCT/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch58_1/pred_img.nii.gz')
    supervised_img = nb.load(supervised_file).get_fdata() 
    if save_collection:
        shutil.copy(supervised_file, os.path.join(save_folder, 'supervised_img.nii.gz'))

    # our method (unsupervised), 1 inference
    unsupervised_file = os.path.join('/host/d/projects/denoising/models/unsupervised_gaussian_brainCT/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch61_1/pred_img.nii.gz')
    unsupervised_img = nb.load(unsupervised_file).get_fdata()
    if save_collection:
        shutil.copy(unsupervised_file, os.path.join(save_folder, 'unsupervised_img.nii.gz'))

    # our method (unsupervised), 10 inference
    unsupervised_avg10_file = os.path.join('/host/d/projects/denoising/models/unsupervised_gaussian_brainCT/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch61avg/pred_img_scans10.nii.gz')
    unsupervised_avg10_img = nb.load(unsupervised_avg10_file).get_fdata()
    if save_collection:
        shutil.copy(unsupervised_avg10_file, os.path.join(save_folder, 'unsupervised_avg10_img.nii.gz'))

    # our method (unsupervised), beta = 0, 20 inference
    unsupervised_avg20_file = os.path.join('/host/d/projects/denoising/models/unsupervised_gaussian_brainCT/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch61avg/pred_img_scans20.nii.gz')
    unsupervised_avg20_img = nb.load(unsupervised_avg20_file).get_fdata()
    if save_collection:
        shutil.copy(unsupervised_avg20_file, os.path.join(save_folder, 'unsupervised_avg20_img.nii.gz'))

    # our method (distilled DDPM)
    distilled_file = os.path.join('/host/d/projects/denoising/models/distill_brainCT/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch164_1/pred_img.nii.gz')
    distilled_img = nb.load(distilled_file).get_fdata()
    if save_collection:
        shutil.copy(distilled_file, os.path.join(save_folder, 'distilled_img.nii.gz'))


    ## calculate metrics
    # MAE
    vmin = 0
    vmax = 100
    mae_condition, mae_condition_std = calc_mae_with_ref_window(condition_img, gt_img, vmin, vmax)
    mae_noise2noise, mae_noise2noise_std = calc_mae_with_ref_window(noise2noise_img, gt_img, vmin, vmax)
    mae_noise2score, mae_noise2score_std = calc_mae_with_ref_window(noise2score_img, gt_img, vmin, vmax)
    mae_ddm2_first, mae_ddm2_first_std = calc_mae_with_ref_window(ddm2_first_img, gt_img, vmin, vmax)
    mae_ddm2_final, mae_ddm2_final_std = calc_mae_with_ref_window(ddm2_final_img, gt_img, vmin, vmax)
    mae_supervised, mae_supervised_std = calc_mae_with_ref_window(supervised_img, gt_img, vmin, vmax)
    mae_unsupervised, mae_unsupervised_std = calc_mae_with_ref_window(unsupervised_img, gt_img, vmin, vmax)
    mae_unsupervised_avg10, mae_unsupervised_avg10_std = calc_mae_with_ref_window(unsupervised_avg10_img, gt_img, vmin, vmax)
    mae_unsupervised_avg20, mae_unsupervised_avg20_std = calc_mae_with_ref_window(unsupervised_avg20_img, gt_img, vmin, vmax)
    mae_distilled, mae_distilled_std = calc_mae_with_ref_window(distilled_img, gt_img, vmin, vmax)
   
    
    # # SSIM
    ssim_condition, ssim_condition_std = calc_ssim_with_ref_window(condition_img, gt_img, vmin, vmax)
    ssim_noise2noise, ssim_noise2noise_std = calc_ssim_with_ref_window(noise2noise_img, gt_img, vmin, vmax)
    ssim_noise2score, ssim_noise2score_std = calc_ssim_with_ref_window(noise2score_img, gt_img, vmin, vmax)
    ssim_ddm2_first, ssim_ddm2_first_std = calc_ssim_with_ref_window(ddm2_first_img, gt_img, vmin, vmax)
    ssim_ddm2_final, ssim_ddm2_final_std = calc_ssim_with_ref_window(ddm2_final_img, gt_img, vmin, vmax)
    ssim_supervised, ssim_supervised_std = calc_ssim_with_ref_window(supervised_img, gt_img, vmin, vmax)
    ssim_unsupervised, ssim_unsupervised_std = calc_ssim_with_ref_window(unsupervised_img, gt_img, vmin, vmax)
    ssim_unsupervised_avg10, ssim_unsupervised_avg10_std = calc_ssim_with_ref_window(unsupervised_avg10_img, gt_img, vmin, vmax)
    ssim_unsupervised_avg20, ssim_unsupervised_avg20_std = calc_ssim_with_ref_window(unsupervised_avg20_img, gt_img, vmin, vmax)
    ssim_distilled, ssim_distilled_std = calc_ssim_with_ref_window(distilled_img, gt_img, vmin, vmax)
    

    # # lpips
    lpips_condition, _ = calc_lpips(condition_img, gt_img, vmin, vmax)
    lpips_noise2noise, _ = calc_lpips(noise2noise_img, gt_img, vmin, vmax)
    lpips_noise2score, _ = calc_lpips(noise2score_img, gt_img, vmin, vmax)
    lpips_ddm2_first, _ = calc_lpips(ddm2_first_img, gt_img, vmin, vmax)
    lpips_ddm2_final, _ = calc_lpips(ddm2_final_img, gt_img, vmin, vmax)
    lpips_supervised, _ = calc_lpips(supervised_img, gt_img, vmin, vmax)
    lpips_unsupervised, _ = calc_lpips(unsupervised_img, gt_img, vmin, vmax)
    lpips_unsupervised_avg10, _ = calc_lpips(unsupervised_avg10_img, gt_img, vmin, vmax)
    lpips_unsupervised_avg20, _ = calc_lpips(unsupervised_avg20_img, gt_img, vmin, vmax)
    lpips_distilled, _ = calc_lpips(distilled_img, gt_img, vmin, vmax)

    print('condition: ', mae_condition, ssim_condition, lpips_condition)
    print('noise2noise: ', mae_noise2noise, ssim_noise2noise, lpips_noise2noise)
    print('noise2score: ', mae_noise2score, ssim_noise2score, lpips_noise2score)
    print('ddm2_first: ', mae_ddm2_first, ssim_ddm2_first, lpips_ddm2_first)
    print('ddm2_final: ', mae_ddm2_final, ssim_ddm2_final, lpips_ddm2_final)
    print('supervised: ', mae_supervised, ssim_supervised, lpips_supervised)
    print('unsupervised: ', mae_unsupervised, ssim_unsupervised, lpips_unsupervised)
    print('unsupervised_avg10: ', mae_unsupervised_avg10, ssim_unsupervised_avg10, lpips_unsupervised_avg10)
    print('unsupervised_avg20: ', mae_unsupervised_avg20, ssim_unsupervised_avg20, lpips_unsupervised_avg20)
    print('distilled: ', mae_distilled, ssim_distilled, lpips_distilled)

    results.append([patient_id, patient_subid, random_n,
                    mae_condition,mae_noise2noise,mae_noise2score,mae_ddm2_first, mae_ddm2_final, mae_supervised, mae_unsupervised, mae_unsupervised_avg10, mae_unsupervised_avg20, mae_distilled,
                    ssim_condition,ssim_noise2noise,ssim_noise2score,ssim_ddm2_first, ssim_ddm2_final, ssim_supervised, ssim_unsupervised, ssim_unsupervised_avg10, ssim_unsupervised_avg20, ssim_distilled,
                    lpips_condition,lpips_noise2noise,lpips_noise2score,lpips_ddm2_first, lpips_ddm2_final, lpips_supervised, lpips_unsupervised, lpips_unsupervised_avg10, lpips_unsupervised_avg20, lpips_distilled
                    ])

    dd = pd.DataFrame(results, columns = ['patient_id','patient_subid','random_n',
    'mae_condition','mae_noise2noise','mae_noise2score','mae_ddm2_first','mae_ddm2_final','mae_supervised','mae_unsupervised','mae_unsupervised_avg10','mae_unsupervised_avg20','mae_distilled',
    'ssim_condition','ssim_noise2noise','ssim_noise2score','ssim_ddm2_first','ssim_ddm2_final','ssim_supervised','ssim_unsupervised','ssim_unsupervised_avg10','ssim_unsupervised_avg20','ssim_distilled',
    'lpips_condition','lpips_noise2noise','lpips_noise2score','lpips_ddm2_first','lpips_ddm2_final','lpips_supervised','lpips_unsupervised','lpips_unsupervised_avg10','lpips_unsupervised_avg20','lpips_distilled'
    ])  
    dd.to_excel('/host/d/projects/denoising/results/brainCT_results.xlsx', index = False)

00019591 0000455445 0
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /opt/conda/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/pytho

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /opt/conda/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/pytho

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /opt/conda/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/python3.10/site-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /opt/conda/lib/pytho

In [9]:
# calculate mean
## metric calculations
results = []
for i in range(0,n.shape[0]):
    patient_id = patient_id_list[n[i]]
    patient_subid = patient_subid_list[n[i]]
    random_n = random_num_list[n[i]]
    print(patient_id, patient_subid, random_n)

    # reference image
    gt_file = os.path.join('/host/d/projects/denoising/models/unsupervised_gaussian_brainCT/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch61_1/gt_img.nii.gz')
    gt_img = nb.load(gt_file).get_fdata()

    # noisy image
    condition_file = os.path.join('/host/d/projects/denoising/models/unsupervised_gaussian_brainCT/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch61_1/condition_img.nii.gz')
    condition_img = nb.load(condition_file).get_fdata()

    # noise2noise
    noise2noise_file = os.path.join('/host/d/projects/denoising/models/noise2noise_brainCT/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch78/pred_img.nii.gz')
    noise2noise_img = nb.load(noise2noise_file).get_fdata() 

    # noise2score
    noise2score_file = os.path.join('/host/d/projects/denoising/models/noise2score_brainCT/pred_images', patient_id, patient_subid,'random_'+str(random_n), 'epoch100/pred_img.nii.gz')
    noise2score_img = nb.load(noise2score_file).get_fdata()


    ## calculate main
    x,y = 256,256
    mean_gt = np.mean(np.clip(gt_img[x-50: x+50, y-50: y+50, 20:40],0,100))
    mean_condition = np.mean(np.clip(condition_img[x-50: x+50, y-50: y+50, 20:40],0,100))
    mean_noise2noise = np.mean(np.clip(noise2noise_img[x-50: x+50, y-50: y+50, 20:40],0,100))
    mean_noise2score = np.mean(np.clip(noise2score_img[x-50: x+50, y-50: y+50, 20:40],0,100))

    print('mean: gt:', mean_gt, ' condition:', mean_condition, ' noise2noise:', mean_noise2noise, ' noise2score:', mean_noise2score)
    
    results.append([patient_id, patient_subid, random_n,
                    mean_gt, mean_condition, mean_noise2noise, mean_noise2score
                    ])
    dd = pd.DataFrame(results, columns = ['patient_id','patient_subid','random_n',
                                             'mean_gt','mean_condition','mean_noise2noise','mean_noise2score'
                                             ])

    dd.to_excel('/host/d/projects/denoising/results/brainCT_mean_results.xlsx', index = False)

00214841 0000455418 0
mean: gt: 21.754015723196854  condition: 21.89032840789795  noise2noise: 19.03606891879365  noise2score: 15.506743506469727
00105734 0000455323 0
mean: gt: 18.486178371301225  condition: 18.97817980621338  noise2noise: 14.566930114135912  noise2score: 14.303804944152832
00214867 0000455521 0
mean: gt: 32.106963860106056  condition: 32.13290597808838  noise2noise: 30.043341023751566  noise2score: 25.375424999389647
00214877 0000455524 0
mean: gt: 31.27295554296927  condition: 31.359965008850097  noise2noise: 28.43605288487233  noise2score: 25.188118078308104
00214836 0000455414 0
mean: gt: 15.307165243522553  condition: 15.87778719329834  noise2noise: 11.581240975344958  noise2score: 9.508770465698243
00154137 0000455529 0
mean: gt: 32.78842260734797  condition: 32.843813273010255  noise2noise: 29.875630303859552  noise2score: 26.09790611022949
00214901 0000455577 0
mean: gt: 25.038895094831215  condition: 25.23733533203125  noise2noise: 21.86812425697546  noise2sc

KeyboardInterrupt: 